In [1]:
from sentence_transformers import SentenceTransformer
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from unidecode import unidecode
from sklearn.cluster import KMeans
import pandas as pd
import nltk
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
import pyLDAvis
#import pyLDAvis.sklearn


nltk.download('stopwords')
stop_words = set(stopwords.words('english'))


/global/homes/m/masare/.conda/envs/sacct/lib/python3.11/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
2024-07-30 07:37:19.777627: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:479] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-30 07:37:19.798104: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:10575] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-30 07:37:19.798138: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1442] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-07-30 07:37:19.814309: I tensorflow/core/p

In [2]:
df=pd.read_csv('/global/homes/m/masare/ticket-text-analysis/ticket_cleaned.csv')
df.head()

,ID,Created,text_data
0,197129,2023-01-18 10:33:43,103343 add another majorana project user permi...
1,197134,2023-01-18 11:11:14,111114 can't edit qos information 020036 - sys...
2,197136,2023-01-18 11:13:15,111315 save qos information previous year 0200...
3,197138,2023-01-18 11:15:50,"111550 ability sort ""confirm users"" table ""wil..."
4,197145,2023-01-18 12:04:41,120441 m3514 re-allocation 020037 - system (ad...


In [3]:
text_data = df['text_data']
print(text_data)

0       103343 add another majorana project user permi...
1       111114 can't edit qos information 020036 - sys...
2       111315 save qos information previous year 0200...
3       111550 ability sort "confirm users" table "wil...
4       120441 m3514 re-allocation 020037 - system (ad...
                              ...                        
6939    220454 php script unable access file outside w...
6940    110740 feb 1 start date - urjoshi sinha - 14" ...
6941    160924 urjoshi sinha - new hire - request offi...
6942    094317 ergonomic evaluation onsite office - 40...
6943    113311 multiple user need fortinet token 02000...
Name: text_data, Length: 6944, dtype: object


In [4]:
# 1. Load a pretrained Sentence Transformer model
model = SentenceTransformer("all-MiniLM-L6-v2")


In [5]:
model

SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
)

In [6]:
df_val = df.text_data.apply(unidecode).to_numpy()

In [7]:
#df_val

In [8]:
# The sentences to encode
#sentences = [
   # "The weather is lovely today.",
  #  "It's so sunny outside!",
 #   "He drove to the stadium.",
#]

In [9]:
# 2. Calculate embeddings by calling model.encode()
embeddings = model.encode(df_val, show_progress_bar=True)
print(embeddings.shape)
# [6944, 384]


Batches:   0%|          | 0/217 [00:00<?, ?it/s]

(6944, 384)


In [10]:
%%time
# 3. Calculate the embedding similarities
similarities = model.similarity(embeddings, embeddings)
print(similarities)
# tensor([[1.0000, 0.6660, 0.1046],
#         [0.6660, 1.0000, 0.1411],
#         [0.1046, 0.1411, 1.0000]])

tensor([[1.0000, 0.3851, 0.4770,  ..., 0.4060, 0.2057, 0.5297],
        [0.3851, 1.0000, 0.7071,  ..., 0.3993, 0.3835, 0.4294],
        [0.4770, 0.7071, 1.0000,  ..., 0.3741, 0.3331, 0.4956],
        ...,
        [0.4060, 0.3993, 0.3741,  ..., 1.0000, 0.5850, 0.5037],
        [0.2057, 0.3835, 0.3331,  ..., 0.5850, 1.0000, 0.3764],
        [0.5297, 0.4294, 0.4956,  ..., 0.5037, 0.3764, 1.0000]])
CPU times: user 452 ms, sys: 123 ms, total: 575 ms
Wall time: 304 ms


In [11]:
# Perform kmean clustering
num_clusters = 30
clustering_model = KMeans(n_clusters=num_clusters)
clustering_model.fit(similarities)
cluster_assignment = clustering_model.labels_

/global/homes/m/masare/.conda/envs/sacct/lib/python3.11/site-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


In [12]:
clustered_sentences = [[] for i in range(num_clusters)]
for sentence_id, cluster_id in enumerate(cluster_assignment):
    clustered_sentences[cluster_id].append(df_val[sentence_id])

In [13]:
#for i, cluster in enumerate(clustered_sentences):
   # print("Cluster ", i + 1)
  #  print(cluster)
  #  print("")

In [14]:
# Print a limited number of sentences per cluster
#for i, cluster in enumerate(clustered_sentences):
 #   print(f"Cluster {i + 1}")
 #   print(cluster[:5])  # Print only the first 5 sentences in each cluster
  #  print("")



In [15]:

# To see the total number of sentences in each cluster
for i, cluster in enumerate(clustered_sentences):
    print(f"Cluster {i + 1} has {len(cluster)} sentences")
    print("")




Cluster 1 has 308 sentences

Cluster 2 has 208 sentences

Cluster 3 has 314 sentences

Cluster 4 has 175 sentences

Cluster 5 has 268 sentences

Cluster 6 has 339 sentences

Cluster 7 has 338 sentences

Cluster 8 has 254 sentences

Cluster 9 has 214 sentences

Cluster 10 has 136 sentences

Cluster 11 has 97 sentences

Cluster 12 has 159 sentences

Cluster 13 has 297 sentences

Cluster 14 has 260 sentences

Cluster 15 has 194 sentences

Cluster 16 has 134 sentences

Cluster 17 has 191 sentences

Cluster 18 has 226 sentences

Cluster 19 has 326 sentences

Cluster 20 has 228 sentences

Cluster 21 has 72 sentences

Cluster 22 has 295 sentences

Cluster 23 has 303 sentences

Cluster 24 has 318 sentences

Cluster 25 has 232 sentences

Cluster 26 has 227 sentences

Cluster 27 has 165 sentences

Cluster 28 has 248 sentences

Cluster 29 has 340 sentences

Cluster 30 has 78 sentences



In [16]:
text_data = df_val.tolist()

In [17]:
# Preprocessing function
def preprocess(text):
    text = text.lower()
    text = ''.join([char for char in text if char.isalnum() or char.isspace()])
    tokens = text.split()
    tokens = [word for word in tokens if word not in stop_words]
    return ' '.join(tokens)


In [18]:
# Preprocess the text data
preprocessed_data = [preprocess(text) for text in text_data]




In [19]:
# Vectorize the text data
vectorizer = CountVectorizer(max_df=0.9, min_df=2, stop_words='english')
text_vectorized = vectorizer.fit_transform(preprocessed_data)


In [20]:
# Apply LDA for topic modeling
num_topics = 30  # Number of topics
lda_model = LatentDirichletAllocation(n_components=num_topics, random_state=42)
lda_model.fit(text_vectorized)


LatentDirichletAllocation(n_components=30, random_state=42)

In [21]:
# Get the topics
def get_topics(model, vectorizer, top_n=10):
    topics = {}
    for idx, topic in enumerate(model.components_):
        topics[f'Topic {idx}'] = [vectorizer.get_feature_names_out()[i] for i in topic.argsort()[:-top_n - 1:-1]]
    return topics


topics = get_topics(lda_model, vectorizer)
for topic, words in topics.items():
    print(f"{topic}: {', '.join(words)}")


Topic 0: file, hp, tape, way, small, bundle, data, nersc, better, htar
Topic 1: quota, file, space, directory, home, used, inode, project, scratch, storage
Topic 2: debug1, debug2, debug3, key, type, host, file, server, packet, hostname
Topic 3: laurie, srun, permission, directory, job, node, error, run, step, best
Topic 4: vasp, license, phillip, thomas, nersc, email, access, users, group, confirmation
Topic 5: job, time, node, error, queue, sbatch, hour, 00, lisa, thanks
Topic 6: justin, nick, globus, thanks, perlmutter, transfer, nersc, nicholas, tyler, data
Topic 7: woosun, yang, regard, michael, ticket, robert, nersc, request, 2023, report
Topic 8: reservation, node, nersc, rebecca, request, time, note, work, hartmanbaker, perlmutter
Topic 9: job, run, node, error, time, sbatch, srun, cpu, issue, phillip
Topic 10: error, erik, time, file, module, tr, johannes, code, install, joseph
Topic 11: file, tape, root, request, best, use, make, hp, result, divij
Topic 12: reply, thanks, err

In [22]:


import pandas as pd
import nltk
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis
from sklearn.feature_extraction.text import CountVectorizer




In [23]:
# Prepare the pyLDAvis visualization
#import pyLDAvis.sklearn
import pyLDAvis.lda_model
pyLDAvis.enable_notebook()
lda_vis_data = pyLDAvis.lda_model.prepare(lda_model, text_vectorized, vectorizer)


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

In [24]:
# Display the visualization
pyLDAvis.display(lda_vis_data)


In [25]:
import json


# Convert the prepared data to JSON
lda_vis_json = pyLDAvis.save_json(lda_vis_data)


# Load the JSON data
lda_vis_dict = json.loads(lda_vis_json)


# Customize the topic colors
topic_colors = ["#FF0000", "#00FF00", "#0000FF", "#FF00FF", "#00FFFF", "#FFFF00", "#FF8000", "#8000FF", "#00FF80", "#80FF00"]
for i, color in enumerate(topic_colors):
    lda_vis_dict['topic']['colors'][str(i+1)] = color


# Save the modified JSON data back
with open("lda_vis_custom.json", "w") as f:
    json.dump(lda_vis_dict, f)


# Load the modified JSON data and display the visualization
pyLDAvis.display(pyLDAvis.display_data(lda_vis_dict))




TypeError: save_json() missing 1 required positional argument: 'fileobj'

In [ ]:
#Semantic Textual Similarity
# Two lists of sentences
sentences1 = [
    "Scratch purge extension",
    "new allocation year",
    "python not working",
    "I like cheese",
]



In [ ]:
sentences2 = [
    "authentication",
    "no username",
    "mfa not working",
    
]

In [ ]:
sentences2 = [
    "my python notebook won't run",
    "new account",
    "mfa not working",
    "I hate cheese",
]

In [ ]:
# Compute embeddings for both lists
embeddings1 = model.encode(sentences1)
embeddings2 = model.encode(sentences2)


In [ ]:
# Compute cosine similarities
similarities2 = model.similarity(embeddings1, embeddings2)

In [ ]:
# Output the pairs with their score
for idx_i, sentence1 in enumerate(sentences1):
    print(sentence1)
    for idx_j, sentence2 in enumerate(sentences2):
        print(f" - {sentence2: <30}: {similarities2[idx_i][idx_j]:.4f}")
#he SentenceTransformer.similarity method returns a 3x3 matrix with the respective cosine similarity scores for all possible pairs between embeddings1 and embeddings2.

In [ ]:
####             BERTopic
#Install these packages in terminal
#pip install bertopic

# Choose an embedding backend
#pip install bertopic[flair,gensim,spacy,use]

# Topic modeling with images
#pip install bertopic[vision]

In [ ]:
titles = df['text_data']

In [ ]:
#text_data[0]

In [ ]:
#Sentence Splitter
from nltk.tokenize import sent_tokenize, word_tokenize
import nltk
nltk.download('punkt')
sentences = [sent_tokenize(text_data) for text_data in text_data]
sentences = [sentence for doc in sentences for sentence in doc]

In [ ]:
from umap import UMAP

umap_model = UMAP(n_neighbors=15, n_components=5, min_dist=0.0, metric='cosine', random_state=42)

In [ ]:
#Controlling Number of Topics
from hdbscan import HDBSCAN

hdbscan_model = HDBSCAN(min_cluster_size=150, metric='euclidean', cluster_selection_method='eom', prediction_data=True)


In [ ]:
#Improving Default Representation
from sklearn.feature_extraction.text import CountVectorizer
vectorizer_model = CountVectorizer(stop_words="english", min_df=2, ngram_range=(1, 2))

In [ ]:
from langchain.chains.question_answering import load_qa_chain
from langchain.llms import OpenAI
chain = load_qa_chain(OpenAI(temperature=0), chain_type="stuff")


In [ ]:
#Additional Representations
import openai
from bertopic.representation import KeyBERTInspired, MaximalMarginalRelevance, OpenAI, PartOfSpeech

# KeyBERT
keybert_model = KeyBERTInspired()

# Part-of-Speech
pos_model = PartOfSpeech("en_core_web_sm")

# MMR
mmr_model = MaximalMarginalRelevance(diversity=0.3)

# GPT-3.5
prompt = """
I have a topic that contains the following documents:
[DOCUMENTS]
The topic is described by the following keywords: [KEYWORDS]

Based on the information above, extract a short but highly descriptive topic label of at most 5 words. Make sure it is in the following format:
topic: <topic label>
"""
client = openai.OpenAI(api_key="sk-...")
openai_model = OpenAI(client, model="gpt-3.5-turbo", exponential_backoff=True, chat=True, prompt=prompt)

# All representation models
representation_model = {
    "KeyBERT": keybert_model,
    # "OpenAI": openai_model,  # Uncomment if you will use OpenAI
    "MMR": mmr_model,
    "POS": pos_model
}